In [5]:
# Importing necessary libraries
import os
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16, VGG19, ResNet50, InceptionV3, Xception
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

# Setting random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Define constants
NUM_CLASSES = 2
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
TRAIN_DIR = 'train'
VAL_DIR = 'val'
TEST_DIR = 'test'
TRAIN_SPLIT = 0.7
VAL_SPLIT = 0.1
TEST_SPLIT = 0.2
test_samples = 280

# Define the base directory where the image data is located
BASE_DIR = r"C:\Users\mohit\Downloads\DIP Assignment 2\Data"

# Set the paths for training, validation, and test data
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
VAL_DIR = os.path.join(BASE_DIR, 'val')
TEST_DIR = os.path.join(BASE_DIR, 'test')

# Load and preprocess the data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(TRAIN_DIR,target_size=IMG_SIZE,batch_size=BATCH_SIZE,class_mode='categorical')

val_generator = val_datagen.flow_from_directory(VAL_DIR,target_size=IMG_SIZE,batch_size=BATCH_SIZE,class_mode='categorical')

test_generator = test_datagen.flow_from_directory(TEST_DIR,target_size=IMG_SIZE,batch_size=BATCH_SIZE,class_mode='categorical',shuffle=False)

# Define function for creating transfer learning models
def create_transfer_model(base_model, num_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Define Xception model
xception_base = Xception(include_top=False, weights='imagenet', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
xception_model = create_transfer_model(xception_base, NUM_CLASSES)

# Compile the models
optimizer = Adam(learning_rate=0.0001)
xception_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define checkpoint callback to save the best model during training
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

xception_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

xception_history = xception_model.fit_generator(train_generator, steps_per_epoch=train_generator.n // train_generator.batch_size, epochs=5, validation_data=val_generator, validation_steps=val_generator.n // val_generator.batch_size, callbacks=[checkpoint])

xception_scores = xception_model.evaluate_generator(test_generator, steps=test_samples // BATCH_SIZE)
print("Xception Test Loss:", xception_scores[0])
print("Xception Test Accuracy:", xception_scores[1])

# Get the number of test samples
num_test_samples = test_generator.n

# Calculate the number of steps for prediction
num_prediction_steps = num_test_samples // test_generator.batch_size + 1

# Generate predictions for all test samples
xception_predictions = xception_model.predict_generator(test_generator, steps=num_prediction_steps, verbose=1)

# Convert predictions to class labels
xception_predicted_labels = np.argmax(xception_predictions, axis=1)

# Get true class labels
true_labels = test_generator.classes

# Calculate classification report
xception_report = classification_report(true_labels, xception_predicted_labels)

print("Xception Classification Report:")
print(xception_report)

Found 980 images belonging to 2 classes.
Found 140 images belonging to 2 classes.
Found 280 images belonging to 2 classes.


C:\Users\mohit\AppData\Local\Temp\ipykernel_33056\3483214989.py:72: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  xception_history = xception_model.fit_generator(train_generator, steps_per_epoch=train_generator.n // train_generator.batch_size, epochs=5, validation_data=val_generator, validation_steps=val_generator.n // val_generator.batch_size, callbacks=[checkpoint])


Epoch 1/5
30/30 [==============================] - 186s 6s/step - loss: 0.1842 - accuracy: 0.9462 - val_loss: 0.1311 - val_accuracy: 0.9219
Epoch 2/5
30/30 [==============================] - 193s 6s/step - loss: 0.0225 - accuracy: 0.9926 - val_loss: 0.1422 - val_accuracy: 0.9297
Epoch 3/5
30/30 [==============================] - 190s 6s/step - loss: 0.0069 - accuracy: 0.9989 - val_loss: 0.0426 - val_accuracy: 0.9922
Epoch 4/5
30/30 [==============================] - 192s 6s/step - loss: 0.0070 - accuracy: 0.9979 - val_loss: 0.0151 - val_accuracy: 0.9922
Epoch 5/5
30/30 [==============================] - 192s 6s/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0074 - val_accuracy: 0.9922


C:\Users\mohit\AppData\Local\Temp\ipykernel_33056\3483214989.py:74: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  xception_scores = xception_model.evaluate_generator(test_generator, steps=test_samples // BATCH_SIZE)


Xception Test Loss: 0.012623521499335766
Xception Test Accuracy: 0.9921875


C:\Users\mohit\AppData\Local\Temp\ipykernel_33056\3483214989.py:85: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  xception_predictions = xception_model.predict_generator(test_generator, steps=num_prediction_steps, verbose=1)


9/9 [==============================] - 11s 1s/step
Xception Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       140
           1       1.00      0.99      0.99       140

    accuracy                           0.99       280
   macro avg       0.99      0.99      0.99       280
weighted avg       0.99      0.99      0.99       280



In [2]:
import matplotlib.pyplot as plt

xception_history = xception_model.fit(train_generator, steps_per_epoch=train_generator.n // train_generator.batch_size, epochs=3)

# Plot the training loss curve
plt.plot(xception_history.history['loss'])
plt.title('xception Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

NameError: name 'xception_model' is not defined

In [ ]:
plt.plot(xception_history.history['accuracy'])
plt.title('xception Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()